In [3]:
!pip install SimpleITK

In [21]:
img_path= r"D:\vestibularSchwannoma\AIIMsInfDataset\ImageNiftis\GKPFX2671\vs_gk_0000.nii.gz"
seg_path= r"D:\vestibularSchwannoma\AIIMsInfDataset\Outputs\GKPFX2671\mask.nii.gz"

import nibabel as nib
img= nib.load(img_path).get_fdata()
seg= nib.load(seg_path).get_fdata()

In [22]:
import SimpleITK as sitk

sitk_image = sitk.GetImageFromArray(img)  
sitk_mask  = sitk.GetImageFromArray(seg)

In [23]:
from radiomics import featureextractor

extractor = featureextractor.RadiomicsFeatureExtractor()
selected_feature_names = ['original_shape_Maximum2DDiameterRow','original_shape_SurfaceArea','original_glszm_LargeAreaHighGrayLevelEmphasis','original_shape_SurfaceVolumeRatio','original_shape_MajorAxisLength','original_shape_Maximum3DDiameter']

# Directly pass the SimpleITK images
features = extractor.execute(sitk_image, sitk_mask, label=1)

extracted_features2 = {
    key: float(features[key]) if hasattr(features[key], "item") else features[key]
    for key in selected_feature_names
    if key in features
}

extracted_features2

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


{'original_shape_Maximum2DDiameterRow': 61.09828148156051,
 'original_shape_SurfaceArea': 4257.442431968991,
 'original_glszm_LargeAreaHighGrayLevelEmphasis': 519659.133611691,
 'original_shape_SurfaceVolumeRatio': 0.3641692863613079,
 'original_shape_MajorAxisLength': 55.63102603003085,
 'original_shape_Maximum3DDiameter': 67.21606950722423}

In [24]:
extracted_features1, extracted_features2

({'original_shape_Maximum2DDiameterRow': 57.245087125446844,
  'original_shape_SurfaceArea': 4795.074468531482,
  'original_glszm_LargeAreaHighGrayLevelEmphasis': 3242937.6816109424,
  'original_shape_SurfaceVolumeRatio': 0.26990934499628394,
  'original_shape_MajorAxisLength': 51.890289371136184,
  'original_shape_Maximum3DDiameter': 57.253820833198546},
 {'original_shape_Maximum2DDiameterRow': 61.09828148156051,
  'original_shape_SurfaceArea': 4257.442431968991,
  'original_glszm_LargeAreaHighGrayLevelEmphasis': 519659.133611691,
  'original_shape_SurfaceVolumeRatio': 0.3641692863613079,
  'original_shape_MajorAxisLength': 55.63102603003085,
  'original_shape_Maximum3DDiameter': 67.21606950722423})

In [42]:
import numpy as np
val = np.log1p(extracted_features2['original_shape_Maximum2DDiameterRow']) 
print(extracted_features2['original_shape_Maximum2DDiameterRow'], val)
val = np.log1p(extracted_features2['original_shape_SurfaceArea']) 
print(extracted_features2['original_shape_SurfaceArea'], val)
val = np.log1p(extracted_features2['original_glszm_LargeAreaHighGrayLevelEmphasis']) 
print(extracted_features2['original_glszm_LargeAreaHighGrayLevelEmphasis'], val)
val = np.log1p(extracted_features2['original_shape_SurfaceVolumeRatio']) 
print(extracted_features2['original_shape_SurfaceVolumeRatio'], val)
val = np.log1p(extracted_features2['original_shape_MajorAxisLength']) 
print(extracted_features2['original_shape_MajorAxisLength'], val)
val = np.log1p(extracted_features2['original_shape_Maximum3DDiameter']) 
print(extracted_features2['original_shape_Maximum3DDiameter'], val)

61.09828148156051 4.128718315151695
4257.442431968991 8.356658746107465
519659.133611691 13.160930287666785
0.3641692863613079 0.3105456619557952
55.63102603003085 4.036556998097391
67.21606950722423 4.222680160367696


In [44]:
def radiomics_features(img_tensor, seg_tensor):
    import SimpleITK as sitk
    from radiomics import featureextractor

    img_tensor= img_tensor.squeeze().squeeze()
    seg_tensor= seg_tensor.squeeze().squeeze()

    sitk_image = sitk.GetImageFromArray(img_tensor.numpy())  
    sitk_mask  = sitk.GetImageFromArray(seg_tensor.numpy())

    extractor = featureextractor.RadiomicsFeatureExtractor()
    selected_feature_names = ['original_shape_Maximum2DDiameterRow','original_shape_SurfaceArea','original_glszm_LargeAreaHighGrayLevelEmphasis','original_shape_SurfaceVolumeRatio','original_shape_MajorAxisLength','original_shape_Maximum3DDiameter']

    # Directly pass the SimpleITK images
    features = extractor.execute(sitk_image, sitk_mask, label=1)

    extracted_features = {
        key: float(np.log1p(features[key])) if hasattr(features[key], "item") else np.log1p(features[key])
        for key in selected_feature_names
        if key in features
    }

    return extracted_features

In [48]:
img_path= r"D:\vestibularSchwannoma\AIIMsInfDataset\ImageNiftis\GKPFX2671\vs_gk_0000.nii.gz"
seg_path= r"D:\vestibularSchwannoma\AIIMsInfDataset\Outputs\GKPFX2671\mask.nii.gz"

import nibabel as nib
img= nib.load(img_path).get_fdata()
seg= nib.load(seg_path).get_fdata()

import torch
img_tensor= torch.from_numpy(img)
seg_tensor= torch.from_numpy(seg)

feat1= radiomics_features(img_tensor, seg_tensor)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [49]:
img_path= r"D:\vestibularSchwannoma\AIIMsInfDataset\ImageNiftis\GK2214\vs_gk_0000.nii.gz"
seg_path= r"D:\vestibularSchwannoma\AIIMsInfDataset\Outputs\GK2214\mask.nii.gz"

import nibabel as nib
img= nib.load(img_path).get_fdata()
seg= nib.load(seg_path).get_fdata()

import torch
img_tensor= torch.from_numpy(img)
seg_tensor= torch.from_numpy(seg)

feat2= radiomics_features(img_tensor, seg_tensor)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [50]:
def rad_mse(d1, d2):
    keys = sorted(d1.keys() & d2.keys())
    if not keys:
        raise ValueError("No overlapping keys.")
    diffsq = []
    for k in keys:
        try:
            v1 = float(d1[k])
            v2 = float(d2[k])
        except (TypeError, ValueError):
            raise TypeError(f"Non-numeric value at key '{k}': {d1[k]} vs {d2[k]}")
        diffsq.append((v1 - v2) ** 2)
    return sum(diffsq) / len(diffsq)

In [53]:
rad_mse(feat1, feat2)

0.567623403073246

In [34]:
import numpy as np

def normalize_features_zscore(feat_dict):
    values = np.array(list(feat_dict.values()), dtype=float)
    mean, std = values.mean(), values.std()
    if std == 0:
        scaled = np.zeros_like(values)
    else:
        scaled = (values - mean) / std
    return dict(zip(feat_dict.keys(), scaled))

normalize_features_zscore(extracted_features2)

{'original_shape_Maximum2DDiameterRow': -0.4514781767875833,
 'original_shape_SurfaceArea': -0.4297737611585581,
 'original_glszm_LargeAreaHighGrayLevelEmphasis': 2.2359972338080514,
 'original_shape_SurfaceVolumeRatio': -0.4517923069844098,
 'original_shape_MajorAxisLength': -0.45150645463620925,
 'original_shape_Maximum3DDiameter': -0.4514465342412912}

In [52]:
feat1, feat2

({'original_shape_Maximum2DDiameterRow': 4.128718315151695,
  'original_shape_SurfaceArea': 8.356658746107465,
  'original_glszm_LargeAreaHighGrayLevelEmphasis': 13.160930287666785,
  'original_shape_SurfaceVolumeRatio': 0.3105456619557952,
  'original_shape_MajorAxisLength': 4.036556998097391,
  'original_shape_Maximum3DDiameter': 4.222680160367696},
 {'original_shape_Maximum2DDiameterRow': 4.064659747694846,
  'original_shape_SurfaceArea': 8.475553043243599,
  'original_glszm_LargeAreaHighGrayLevelEmphasis': 14.991990477189312,
  'original_shape_SurfaceVolumeRatio': 0.23894551603000155,
  'original_shape_MajorAxisLength': 3.968219756250976,
  'original_shape_Maximum3DDiameter': 4.064809683997616})

In [31]:
(57.245087125446844-61.09828148156051)**2 + (4795.074468531482-4257.442431968991)**2 + (3242937.6816109424-519659.133611691)**2 + (0.26990934499628394-0.3641692863613079)**2 + (51.890289371136184-55.63102603003085)**2 + (57.253820833198546-67.21606950722423)**2

7416246339169.212

In [33]:
7416246339169.212/6

1236041056528.202